In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras import backend as K
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

Using TensorFlow backend.


In [2]:
embeddings_index = {}
path = '/Users/pinarayaz/Jupyter/NLP/data/wiki_embeddings_word2vec.txt'
f = open(path, encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [3]:
#https://stackoverflow.com/a/45305384
def pre(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
def rec(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
def f1(y_true, y_pred):
    precision = pre(y_true, y_pred)
    recall = rec(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## tremo dataset

In [4]:
#import tremo dataset
tremopath = '/Users/pinarayaz/Jupyter/NLP/data/tremo_preprocessed.csv'
tremo_df = pd.read_csv(tremopath)
tremo_df.head()

,Entry,ValidatedEmotion
0,yeni gün bir mutlu,Happy
1,gece ol sokak geç kork,Fear
2,gerçek hayal,Sadness
3,arkadaş kaybet üz,Sadness
4,insan çıkar ol tiksin,Disgust


In [5]:
entries = tremo_df['Entry'].values.tolist()
max_length = max([len(s.split()) for s in entries])
print("Max length:", max_length)

#vectorize texts into 2d integer tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(entries)
sequences = tokenizer.texts_to_sequences(entries)

#pad sequences
word_index = tokenizer.word_index
print("Unique tokens: ", len(word_index))

entry_pad = pad_sequences(sequences, maxlen=max_length)
emotion = tremo_df['ValidatedEmotion'].values

lab_enc = preprocessing.LabelEncoder()
emotion = lab_enc.fit_transform(emotion)

print("Shape of entry tensor:", entry_pad.shape)
print("Shape of emotion tensor:", emotion.shape)

Max length: 55
Unique tokens:  7925
Shape of entry tensor: (25989, 55)
Shape of emotion tensor: (25989,)


In [6]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(num_words)

7926


In [7]:
#define model
model = Sequential()
embedding_layer = Embedding(num_words,
                           EMBEDDING_DIM,
                           embeddings_initializer=Constant(embedding_matrix),
                           input_length=max_length,
                           trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='sigmoid'))

#optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', pre, rec, f1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
#split the data
VALIDATION_SPLIT = 0.2

indices = np.arange(entry_pad.shape[0])
#np.random.shuffle(indices)
entry_pad = entry_pad[indices]
emotion = emotion[indices]
num_validation_samples = int(VALIDATION_SPLIT * entry_pad.shape[0])

# Convert labels to categorical one-hot encoding
emotion = to_categorical(emotion, num_classes=6)

X_train_pad = entry_pad[:-num_validation_samples]
y_train = emotion[:-num_validation_samples]
X_test_pad = entry_pad[-num_validation_samples:]
y_test = emotion[-num_validation_samples:]
print("Shape of X_train_pad tensor:", X_train_pad.shape)
print("Shape of y_train tensor:", y_train.shape)
print("Shape of X_test_pad tensor:", X_test_pad.shape)
print("Shape of y_test tensor:", y_test.shape)

Shape of X_train_pad tensor: (20792, 55)
Shape of y_train tensor: (20792, 6)
Shape of X_test_pad tensor: (5197, 55)
Shape of y_test tensor: (5197, 6)


In [9]:
#train the model
model.fit(X_train_pad, y_train, batch_size=64, epochs=50, validation_data=(X_test_pad, y_test), verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 20792 samples, validate on 5197 samples
Epoch 1/50
 - 13s - loss: 0.3176 - acc: 0.8814 - pre: 0.8085 - rec: 0.3904 - f1: 0.5023 - val_loss: 0.2553 - val_acc: 0.9065 - val_pre: 0.8587 - val_rec: 0.5153 - val_f1: 0.6356
Epoch 2/50
 - 12s - loss: 0.2074 - acc: 0.9267 - pre: 0.8904 - rec: 0.6389 - f1: 0.7426 - val_loss: 0.2192 - val_acc: 0.9158 - val_pre: 0.8389 - val_rec: 0.6028 - val_f1: 0.6970
Epoch 3/50
 - 12s - loss: 0.1792 - acc: 0.9343 - pre: 0.8909 - rec: 0.6904 - f1: 0.7768 - val_loss: 0.2038 - val_acc: 0.9221 - val_pre: 0.8486 - val_rec: 0.6404 - val_f1: 0.7265
Epoch 4/50
 - 12s - loss: 0.1646 - acc: 0.9387 - pre: 0.8913 - rec: 0.7202 - f1: 0.7958 - val_loss: 0.1942 - val_acc: 0.9266 - val_pre: 0.8556 - val_rec: 0.6656 - val_f1: 0.7460
Epoch 5/50
 - 12s - loss: 0.1552 - acc: 0.9418 - pre: 0.8936 - rec: 0.7391 - f1: 0.8083 - val_loss: 0.1890 - val_acc: 0.9278 - val_pre: 0.8493 - val_rec: 0.6823 - val_f1: 0.7544
Epoch 6/50
 -

Epoch 46/50
 - 12s - loss: 0.1147 - acc: 0.9555 - pre: 0.9060 - rec: 0.8182 - f1: 0.8594 - val_loss: 0.1742 - val_acc: 0.9352 - val_pre: 0.8371 - val_rec: 0.7535 - val_f1: 0.7921
Epoch 47/50
 - 12s - loss: 0.1143 - acc: 0.9559 - pre: 0.9079 - rec: 0.8189 - f1: 0.8606 - val_loss: 0.1737 - val_acc: 0.9354 - val_pre: 0.8414 - val_rec: 0.7497 - val_f1: 0.7918
Epoch 48/50
 - 12s - loss: 0.1151 - acc: 0.9550 - pre: 0.9031 - rec: 0.8180 - f1: 0.8579 - val_loss: 0.1739 - val_acc: 0.9350 - val_pre: 0.8353 - val_rec: 0.7552 - val_f1: 0.7923
Epoch 49/50
 - 12s - loss: 0.1155 - acc: 0.9552 - pre: 0.9042 - rec: 0.8185 - f1: 0.8587 - val_loss: 0.1726 - val_acc: 0.9354 - val_pre: 0.8387 - val_rec: 0.7529 - val_f1: 0.7925
Epoch 50/50
 - 11s - loss: 0.1140 - acc: 0.9560 - pre: 0.9065 - rec: 0.8216 - f1: 0.8613 - val_loss: 0.1740 - val_acc: 0.9345 - val_pre: 0.8338 - val_rec: 0.7533 - val_f1: 0.7905


In [10]:
#confusion matrix
y_pred = model.predict(X_test_pad)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

[[773  28  48  50  29  20]
 [ 28 612  31  15   5   5]
 [ 34  15 705  37  43  10]
 [ 47  25  40 920  32  39]
 [ 74  16  71 129 802  18]
 [ 31  10  21  78  23 333]]


In [11]:
#get y_pred for meta layer
all_pred = model.predict(entry_pad)
decoded = lab_enc.inverse_transform(all_pred.argmax(axis=1))
print(decoded)
(pd.DataFrame(decoded)).to_csv('predictions.csv', index=False, header=False)

['Happy' 'Fear' 'Sadness' ... 'Fear' 'Anger' 'Surprise']


## twt dataset

In [53]:
#read twt dataset
twtpath = '/Users/pinarayaz/Jupyter/NLP/data/twt_preprocessed.csv'
twt_df = pd.read_csv(twtpath)
twt_df.head()

,Tweet,Label
0,kız nutellayı abartıyo sosyal medya nutella üz...,negative
1,ooooo dedikodu al bi dal :(,negative
2,:(,negative
3,bıkmayacaksın :),positive
4,whatsapp grup gelecek favlasın muhabbet fena :),positive


In [54]:
entries = twt_df['Tweet'].values.astype('U').tolist()
max_length = max([len(s.split()) for s in entries])
print("Max length:", max_length)

#vectorize texts into 2d integer tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(entries)
sequences = tokenizer.texts_to_sequences(entries)

#pad sequences
word_index = tokenizer.word_index
print("Unique tokens: ", len(word_index))

entry_pad = pad_sequences(sequences, maxlen=max_length)
emotion = twt_df['Label'].values.astype('U')

lab_enc = preprocessing.LabelEncoder()
emotion = lab_enc.fit_transform(emotion)

print("Shape of entry tensor:", entry_pad.shape)
print("Shape of emotion tensor:", emotion.shape)

Max length: 28
Unique tokens:  27809
Shape of entry tensor: (32000, 28)
Shape of emotion tensor: (32000,)


In [55]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(num_words)

27810


In [56]:
#define model
model = Sequential()
embedding_layer = Embedding(num_words,
                           EMBEDDING_DIM,
                           embeddings_initializer=Constant(embedding_matrix),
                           input_length=max_length,
                           trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

#optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', pre, rec, f1])

In [57]:
#split the data
VALIDATION_SPLIT = 0.2

indices = np.arange(entry_pad.shape[0])
np.random.shuffle(indices)
entry_pad = entry_pad[indices]
emotion = emotion[indices]
num_validation_samples = int(VALIDATION_SPLIT * entry_pad.shape[0])

X_train_pad = entry_pad[:-num_validation_samples]
y_train = emotion[:-num_validation_samples]
X_test_pad = entry_pad[-num_validation_samples:]
y_test = emotion[-num_validation_samples:]
print("Shape of X_train_pad tensor:", X_train_pad.shape)
print("Shape of y_train tensor:", y_train.shape)
print("Shape of X_test_pad tensor:", X_test_pad.shape)
print("Shape of y_test tensor:", y_test.shape)

Shape of X_train_pad tensor: (25600, 28)
Shape of y_train tensor: (25600,)
Shape of X_test_pad tensor: (6400, 28)
Shape of y_test tensor: (6400,)


In [58]:
#train the model
model.fit(X_train_pad, y_train, batch_size=64, epochs=50, validation_data=(X_test_pad, y_test), verbose=2)

Train on 25600 samples, validate on 6400 samples
Epoch 1/50
 - 10s - loss: 0.6256 - acc: 0.6368 - pre: 0.6346 - rec: 0.6573 - f1: 0.6385 - val_loss: 0.5627 - val_acc: 0.6934 - val_pre: 0.6917 - val_rec: 0.6960 - val_f1: 0.6903
Epoch 2/50
 - 8s - loss: 0.5606 - acc: 0.6986 - pre: 0.7099 - rec: 0.6726 - f1: 0.6869 - val_loss: 0.5318 - val_acc: 0.7113 - val_pre: 0.7208 - val_rec: 0.6889 - val_f1: 0.7009
Epoch 3/50
 - 8s - loss: 0.5338 - acc: 0.7170 - pre: 0.7359 - rec: 0.6805 - f1: 0.7035 - val_loss: 0.5176 - val_acc: 0.7242 - val_pre: 0.7515 - val_rec: 0.6691 - val_f1: 0.7044
Epoch 4/50
 - 8s - loss: 0.5158 - acc: 0.7269 - pre: 0.7464 - rec: 0.6890 - f1: 0.7125 - val_loss: 0.5074 - val_acc: 0.7342 - val_pre: 0.7466 - val_rec: 0.7112 - val_f1: 0.7251
Epoch 5/50
 - 8s - loss: 0.5048 - acc: 0.7325 - pre: 0.7529 - rec: 0.6952 - f1: 0.7193 - val_loss: 0.5016 - val_acc: 0.7348 - val_pre: 0.7522 - val_rec: 0.7035 - val_f1: 0.7235
Epoch 6/50
 - 8s - loss: 0.4953 - acc: 0.7415 - pre: 0.7627 - rec

Epoch 47/50
 - 8s - loss: 0.4176 - acc: 0.7893 - pre: 0.8120 - rec: 0.7517 - f1: 0.7777 - val_loss: 0.4954 - val_acc: 0.7470 - val_pre: 0.7908 - val_rec: 0.6738 - val_f1: 0.7234
Epoch 48/50
 - 8s - loss: 0.4141 - acc: 0.7929 - pre: 0.8169 - rec: 0.7550 - f1: 0.7817 - val_loss: 0.4970 - val_acc: 0.7436 - val_pre: 0.7574 - val_rec: 0.7188 - val_f1: 0.7343
Epoch 49/50
 - 8s - loss: 0.4136 - acc: 0.7927 - pre: 0.8158 - rec: 0.7568 - f1: 0.7825 - val_loss: 0.4998 - val_acc: 0.7491 - val_pre: 0.7914 - val_rec: 0.6770 - val_f1: 0.7261
Epoch 50/50
 - 8s - loss: 0.4152 - acc: 0.7914 - pre: 0.8149 - rec: 0.7521 - f1: 0.7793 - val_loss: 0.4955 - val_acc: 0.7486 - val_pre: 0.7940 - val_rec: 0.6728 - val_f1: 0.7238


In [62]:
#confusion matrix
y_pred = model.predict_classes(X_test_pad)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[2645  563]
 [1046 2146]]


## tweets dataset

In [63]:
#import tweets dataset
tweetspath = '/Users/pinarayaz/Jupyter/NLP/data/tweets_preprocessed.csv'
tweets_df = pd.read_csv(tweetspath)
tweets_df.head()

,Tweet,Sentiment
0,ulan wifi'ye bağ bağ türkcell ınternet paket b...,olumsuz
1,20 dk 1 gb internet 500 mb kaşar türkcell düş ...,olumsuz
2,türkcell superonline reklam kötü bir reklam gör,olumsuz
3,türkcell paha,olumsuz
4,türkcell kaş'ta internet çek,olumsuz


In [64]:
entries = tweets_df['Tweet'].values.tolist()
max_length = max([len(s.split()) for s in entries])
print("Max length:", max_length)

#vectorize texts into 2d integer tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(entries)
sequences = tokenizer.texts_to_sequences(entries)

#pad sequences
word_index = tokenizer.word_index
print("Unique tokens: ", len(word_index))

entry_pad = pad_sequences(sequences, maxlen=max_length)
emotion = tweets_df['Sentiment'].values

lab_enc = preprocessing.LabelEncoder()
emotion = lab_enc.fit_transform(emotion)

print("Shape of entry tensor:", entry_pad.shape)
print("Shape of emotion tensor:", emotion.shape)

Max length: 37
Unique tokens:  25984
Shape of entry tensor: (17289, 37)
Shape of emotion tensor: (17289,)


In [65]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(num_words)

25985


In [66]:
#define model
model = Sequential()
embedding_layer = Embedding(num_words,
                           EMBEDDING_DIM,
                           embeddings_initializer=Constant(embedding_matrix),
                           input_length=max_length,
                           trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='sigmoid'))

#optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', pre, rec, f1])

In [67]:
#split the data
VALIDATION_SPLIT = 0.2

indices = np.arange(entry_pad.shape[0])
np.random.shuffle(indices)
entry_pad = entry_pad[indices]
emotion = emotion[indices]
num_validation_samples = int(VALIDATION_SPLIT * entry_pad.shape[0])

# Convert labels to categorical one-hot encoding
emotion = to_categorical(emotion, num_classes=3)

X_train_pad = entry_pad[:-num_validation_samples]
y_train = emotion[:-num_validation_samples]
X_test_pad = entry_pad[-num_validation_samples:]
y_test = emotion[-num_validation_samples:]
print("Shape of X_train_pad tensor:", X_train_pad.shape)
print("Shape of y_train tensor:", y_train.shape)
print("Shape of X_test_pad tensor:", X_test_pad.shape)
print("Shape of y_test tensor:", y_test.shape)

Shape of X_train_pad tensor: (13832, 37)
Shape of y_train tensor: (13832, 3)
Shape of X_test_pad tensor: (3457, 37)
Shape of y_test tensor: (3457, 3)


In [68]:
#train the model
model.fit(X_train_pad, y_train, batch_size=64, epochs=50, validation_data=(X_test_pad, y_test), verbose=2)

Train on 13832 samples, validate on 3457 samples
Epoch 1/50
 - 8s - loss: 0.6150 - acc: 0.6736 - pre: 0.5375 - rec: 0.1995 - f1: 0.2860 - val_loss: 0.5630 - val_acc: 0.7206 - val_pre: 0.6732 - val_rec: 0.3141 - val_f1: 0.4262
Epoch 2/50
 - 6s - loss: 0.5638 - acc: 0.7170 - pre: 0.6365 - rec: 0.3525 - f1: 0.4517 - val_loss: 0.5359 - val_acc: 0.7401 - val_pre: 0.6901 - val_rec: 0.4024 - val_f1: 0.5066
Epoch 3/50
 - 5s - loss: 0.5414 - acc: 0.7328 - pre: 0.6543 - rec: 0.4200 - f1: 0.5103 - val_loss: 0.5191 - val_acc: 0.7501 - val_pre: 0.6834 - val_rec: 0.4675 - val_f1: 0.5540
Epoch 4/50
 - 6s - loss: 0.5282 - acc: 0.7414 - pre: 0.6679 - rec: 0.4479 - f1: 0.5350 - val_loss: 0.5111 - val_acc: 0.7576 - val_pre: 0.6992 - val_rec: 0.4811 - val_f1: 0.5691
Epoch 5/50
 - 6s - loss: 0.5181 - acc: 0.7494 - pre: 0.6777 - rec: 0.4731 - f1: 0.5559 - val_loss: 0.5060 - val_acc: 0.7587 - val_pre: 0.6964 - val_rec: 0.4915 - val_f1: 0.5753
Epoch 6/50
 - 6s - loss: 0.5094 - acc: 0.7546 - pre: 0.6892 - rec:

Epoch 47/50
 - 5s - loss: 0.4353 - acc: 0.7966 - pre: 0.7435 - rec: 0.5964 - f1: 0.6609 - val_loss: 0.4879 - val_acc: 0.7738 - val_pre: 0.7010 - val_rec: 0.5629 - val_f1: 0.6237
Epoch 48/50
 - 5s - loss: 0.4336 - acc: 0.7992 - pre: 0.7489 - rec: 0.5990 - f1: 0.6648 - val_loss: 0.4883 - val_acc: 0.7728 - val_pre: 0.6980 - val_rec: 0.5635 - val_f1: 0.6228
Epoch 49/50
 - 5s - loss: 0.4326 - acc: 0.8008 - pre: 0.7503 - rec: 0.6038 - f1: 0.6684 - val_loss: 0.4882 - val_acc: 0.7737 - val_pre: 0.7016 - val_rec: 0.5592 - val_f1: 0.6215
Epoch 50/50
 - 5s - loss: 0.4310 - acc: 0.8000 - pre: 0.7491 - rec: 0.6012 - f1: 0.6663 - val_loss: 0.4915 - val_acc: 0.7706 - val_pre: 0.6898 - val_rec: 0.5673 - val_f1: 0.6219


In [69]:
#confusion matrix
y_pred = model.predict(X_test_pad)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

[[ 723  162  275]
 [ 182  452  245]
 [ 234  134 1050]]


## reviews dataset

In [78]:
#import reviews dataset
reviewspath = '/Users/pinarayaz/Jupyter/NLP/data/reviews_preprocessed.csv'
reviews_df = pd.read_csv(reviewspath)
reviews_df.head()

,Review,Rating,Sentiment
0,nicholson harika,4.0,positive
1,mükemmel derece kötü arkadaş sinema bilgi şüph...,5.0,positive
2,mükemmel derece kötü bi film biselerin ol bekl...,1.0,negative
3,beğen anla film film oyun senaryo zayıf film f...,1.5,negative
4,ok harika bir film senaryo fazla ol yüksek per...,5.0,positive


In [79]:
entries = reviews_df['Review'].values.astype('U').tolist()
max_length = max([len(s.split()) for s in entries])
print("Max length:", max_length)

#vectorize texts into 2d integer tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(entries)
sequences = tokenizer.texts_to_sequences(entries)

#pad sequences
word_index = tokenizer.word_index
print("Unique tokens: ", len(word_index))

entry_pad = pad_sequences(sequences, maxlen=max_length)
emotion = reviews_df['Sentiment'].values.astype('U')

lab_enc = preprocessing.LabelEncoder()
emotion = lab_enc.fit_transform(emotion)

print("Shape of entry tensor:", entry_pad.shape)
print("Shape of emotion tensor:", emotion.shape)

Max length: 1827
Unique tokens:  47128
Shape of entry tensor: (34990, 1827)
Shape of emotion tensor: (34990,)


In [80]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(num_words)

47129


In [81]:
#define model
model = Sequential()
embedding_layer = Embedding(num_words,
                           EMBEDDING_DIM,
                           embeddings_initializer=Constant(embedding_matrix),
                           input_length=max_length,
                           trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

#optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', pre, rec, f1])

In [82]:
#split the data
VALIDATION_SPLIT = 0.2

indices = np.arange(entry_pad.shape[0])
np.random.shuffle(indices)
entry_pad = entry_pad[indices]
emotion = emotion[indices]
num_validation_samples = int(VALIDATION_SPLIT * entry_pad.shape[0])

X_train_pad = entry_pad[:-num_validation_samples]
y_train = emotion[:-num_validation_samples]
X_test_pad = entry_pad[-num_validation_samples:]
y_test = emotion[-num_validation_samples:]
print("Shape of X_train_pad tensor:", X_train_pad.shape)
print("Shape of y_train tensor:", y_train.shape)
print("Shape of X_test_pad tensor:", X_test_pad.shape)
print("Shape of y_test tensor:", y_test.shape)

Shape of X_train_pad tensor: (27992, 1827)
Shape of y_train tensor: (27992,)
Shape of X_test_pad tensor: (6998, 1827)
Shape of y_test tensor: (6998,)


In [86]:
#train the model
model.fit(X_train_pad, y_train, batch_size=128, epochs=5, validation_data=(X_test_pad, y_test), verbose=1)

Train on 27992 samples, validate on 6998 samples
Epoch 1/5
27992/27992 [==============================] - 286s 10ms/step - loss: 0.5543 - acc: 0.7269 - pre: 0.7398 - rec: 0.9412 - f1: 0.8277 - val_loss: 0.5328 - val_acc: 0.7481 - val_pre: 0.7510 - val_rec: 0.9591 - val_f1: 0.8417
Epoch 2/5
27992/27992 [==============================] - 299s 11ms/step - loss: 0.5329 - acc: 0.7416 - pre: 0.7582 - rec: 0.9270 - f1: 0.8334 - val_loss: 0.5222 - val_acc: 0.7581 - val_pre: 0.7620 - val_rec: 0.9531 - val_f1: 0.8462
Epoch 3/5
27992/27992 [==============================] - 282s 10ms/step - loss: 0.5204 - acc: 0.7506 - pre: 0.7674 - rec: 0.9238 - f1: 0.8377 - val_loss: 0.5093 - val_acc: 0.7608 - val_pre: 0.7749 - val_rec: 0.9297 - val_f1: 0.8445
Epoch 4/5
27992/27992 [==============================] - 330s 12ms/step - loss: 0.5104 - acc: 0.7587 - pre: 0.7753 - rec: 0.9235 - f1: 0.8422 - val_loss: 0.5024 - val_acc: 0.7672 - val_pre: 0.7807 - val_rec: 0.9300 - val_f1: 0.8481
Epoch 5/5
27992/27992 [

In [87]:
#confusion matrix
y_pred = model.predict_classes(X_test_pad)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[ 854 1221]
 [ 395 4528]]
